In [19]:
#First step - Set up our code

import os, arcpy
from datetime import datetime

os.chdir(r"C:\Users\secuc\ArcGIS_Projects\Project")
print(f"{os.getcwd()}")

mosaicData = "OISSSvs2_2023"
crfMosaic = f"C:\\Users\\secuc\\MyFiles\\CRF\\{mosaicData}.crf"
rasterfolder = r"C:\Users\secuc\MyFiles\OISSS_v2\OISSS_2023"
aGDB = r"C:\Users\secuc\ArcGIS_Projects\Project\OISSS.gdb"

print(f'{mosaicData}\nDone at {datetime.now().strftime("%H:%M:%S")}!')

C:\Users\secuc\ArcGIS_Projects\Project
OISSSvs2_2023
Done at 14:05:18!


In [20]:
#Second Step - Set up parameters & run Create Mosaic.
#Third Step - Add Rasters Tools

updatecs = "NO_CELL_SIZES"
updatebnd = "UPDATE_BOUNDARY"
updateovr = "UPDATE_OVERVIEWS"
maxlevel = ""
maxcs = ""
maxdim = ""
spatialref = ""
inputdatafilter = "*.nc"
subfolder = "SUBFOLDERS"
duplicate = "EXCLUDE_DUPLICATES"
buildpy = "NO_PYRAMIDS"
calcstats = "CALCULATE_STATISTICS"
buildthumb = "NO_THUMBNAILS"
comments = "Add Raster Datasets"
forcesr = ""
estimatestats = "ESTIMATE_STATISTICS"
auxilaryinput = ""
enablepixcache = "NO_PIXEL_CACHE"
cachelocation = ""

arcpy.CreateMosaicDataset_management(f"{aGDB}", mosaicData, arcpy.SpatialReference(3857))

arcpy.management.AddRastersToMosaicDataset(f"{aGDB}\\{mosaicData}", "NetCDF", f"{rasterfolder}", f"{updatecs}", f"{updatebnd}", f"{updateovr}", f"{maxlevel}", f"{maxcs}", f"{maxdim}", f"{spatialref}", f"{inputdatafilter}", f"{subfolder}", f"{duplicate}", f"{buildpy}", f"{calcstats}", f"{buildthumb}", f"{comments}", f"{forcesr}", f"{estimatestats}", f"{auxilaryinput}", f"{enablepixcache}", f"{cachelocation}")

print(f'Done at {datetime.now().strftime("%H:%M:%S")}!')

Done at 14:05:55!


### DO NOT PROCEED WITH NEXT STEPS. THEY CAUSE ERRORS TO OCCUR.

### ****

In [ ]:
#Fourth Step - Create and populate the field to extract the date from the Name. 
#Fifth convert to date.

arcpy.management.AddField(mosaicData, "Time", "TEXT")

with arcpy.da.UpdateCursor(mosaicData, ["Name", "Time"]) as aCursor:
    for aRow in aCursor:
        if "OISSS" in aRow[0]:
            aRow[1] = aRow[0][42:49]
            aCursor.updateRow(aRow)

arcpy.ConvertTimeField_management(f"{mosaicData}\Footprint", "Time", "yyyy-MM-dd HHmmss", "Time_Extracted")

print(f'Done at {datetime.now().strftime("%H:%M:%S")}!')

### Sixth Step - Build the multidimensional information.

arcpy.md.BuildMultidimensionalInfo(f"{mosaicData}_2", "sss", "Time_Extracted")

print("Done!")

#DO NOT USE UNTIL WE FIGURE OUT WHATS WRONG WITH THIS CODE!!!

### Proceed below

### Seventh Step - Convert to CRF
- Make sure to maintain clipping extent!
- Make sure to save in CRF folder!
- Make sure to apply snap raster!

### Eighth Step - Resample 4630 - UpSampling - higher resolutions but worse statistics.
## False Accuracy!!! Best to Downsample

arcpy.management.Resample(crfMosaic, mosaicResam, 4630, "NEAREST")

print(f'Done at {datetime.now().strftime("%H:%M:%S")}!')

In [22]:
#Eighth Step - Resample 27820 - Upsample a little bit, so should be okay. Still low res...
mosaicDataa = ["OISSSvs2_2011", "OISSSvs2_2012", "OISSSvs2_2013", "OISSSvs2_2014", "OISSSvs2_2015", "OISSSvs2_2016", "OISSSvs2_2017", "OISSSvs2_2018", "OISSSvs2_2019", "OISSSvs2_2020", "OISSSvs2_2021", "OISSSvs2_2022", "OISSSvs2_2023"]

for item in mosaicDataa:
    arcpy.management.Resample(f"{item}.crf", f"C:\\Users\\secuc\\MyFiles\\Resampled_27800\\{item}_Resample27.crf", 27820, "NEAREST")

print(f'Done at {datetime.now().strftime("%H:%M:%S")}!')

Done at 14:14:11!


In [ ]:
# Data managing...

arcpy.DeleteField_management(mosaicData, "MO")
arcpy.DeleteField_management(mosaicData, "Time")

print(f'Done at {datetime.now().strftime("%H:%M:%S")}!')